In [21]:
from math import exp
from scipy.optimize import root

In [4]:
def Antonie(T, param):
    '''
    T : Temprature in Centigrade
    param: list of [A, B, C]
    
    This Function Computes saturated Pressure at given Temprature
    '''
    if len(param) != 3:
        raise Exception("param must be list of [A, B, C]")
    
    A = param[0]
    B = param[1]
    C = param[2]

    ln_P_sat = A + B/(T + C)
    P_sat = exp(ln_P_sat)

    return P_sat

In [8]:
def BUBL_P(X,T,all_params):
    '''
    X = list of mollar fractions of liquid like [0.2 ,0.8] or [0.1 0.2 0.7]
        Sumation of X list must be 1.0
    
    T = Temprature in Centigrade

    all_params = list of parameters for Antonie equations
    
    example for all params:
        all_params = [[A1, B1, C1],
                      [A2, B2, C2],
                      [A3, B3, C3]] 
    '''

    if len(X) != len(all_params):
        raise Exception("count of ellements of X and all_params must be equal!")

    if sum(X) != 1.0:
        raise Exception("Summation of all X list must be 1.0")

    P = 0
    for i in range(len(X)):
        xi = X[i]
        param = all_params[i]
        
        P_sat_i = Antonie(T,param)
        
        P += P_sat_i * xi

    Y = []
    for i in range(len(X)):
        xi = X[i]
        param = all_params[i]
        
        P_sat_i = Antonie(T, param)

        yi = (xi * P_sat_i)/P
        Y.append(yi)
    
    return P,Y


In [31]:
def BUBL_T(X,P,all_params):
    """
    X = list of mollar fractions of liquid like [0.2 ,0.8] or [0.1 0.2 0.7]
        Sumation of X list must be 1.0

    P = Pressure in kPa
    
    all_params = list of parameters for Antonie equations

    example for all params:
        all_params = [[A1, B1, C1],
                      [A2, B2, C2],
                      [A3, B3, C3]]
    """
    def func(T):
        return (P - BUBL_P(X, T, all_params)[0])
    
    solve = root(func,20,method='lm' )
    T = solve['x'][0]
    nP,Y = BUBL_P(X,T,all_params)

    return T,Y

In [37]:
def DEW_P(Y, T, all_params):
    '''
    Y = list of mollar fractions of vapor like [0.2 ,0.8] or [0.1 0.2 0.7]
        Sumation of X list must be 1.0
    
    T = Temprature in Centigrade

    all_params = list of parameters for Antonie equations
    
    example for all params:
        all_params = [[A1, B1, C1],
                      [A2, B2, C2],
                      [A3, B3, C3]] 
    ''' 
    if len(Y) != len(all_params):
        raise Exception("count of ellements of Y and all_params must be equal!")

    if sum(Y) != 1.0:
        raise Exception("Summation of all Y list must be 1.0")

    sum_yi_Psat = 0
    for i in range(len(Y)):
        yi = Y[i]
        param = all_params[i]
        
        P_sat_i = Antonie(T, param)
        
        sum_yi_Psat += yi/P_sat_i

    P = 1/sum_yi_Psat

    X = []
    for i in range(len(Y)):
        yi = Y[i]
        param = all_params[i]
        
        P_sat_i = Antonie(T, param)

        xi = (yi * P)/P_sat_i
        X.append(xi)
    
    return P, X

In [40]:
def DEW_T(Y, P, all_params):
    """
    Y = list of mollar fractions of vapor like [0.2 ,0.8] or [0.1 0.2 0.7]
        Sumation of X list must be 1.0

    P = Pressure in kPa
    
    all_params = list of parameters for Antonie equations

    example for all params:
        all_params = [[A1, B1, C1],
                      [A2, B2, C2],
                      [A3, B3, C3]]
    """
    def func(T):
        return (P - DEW_P(Y, T, all_params))

    solve = root(func, 20, method='lm')
    T = solve['x'][0]

    nP,Y = DEW_P(X, T, all_params)

    return T,Y